In [50]:
"""
Author: Zaw
Findin the three most deviations of Predicting the LogP of molecules by using RF
"""
import numpy as np
import pandas as pd
import math
import time
import os
import sys
import warnings
warnings.simplefilter(action='ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from hyperopt import fmin,tpe,hp,STATUS_OK, Trials
import pickle
start = time.time()

In [51]:
# Data path
path = os.getcwd()
dirname = os.path.dirname(path)
data_path = dirname+"/LogP/data/extract_data.csv"
col_list=['LogP','Exp_RT']
lc_df = pd.read_csv(data_path,usecols=col_list)
col_smile_list=['SMILES','LogP','Exp_RT']
lc_smiles_df = pd.read_csv(data_path,usecols=col_smile_list)

# Remove non_retained molecules
# index=lc_df[lc_df['Exp_RT'] < 180].index
# lc_df.drop(lc_df[lc_df['Exp_RT'] < 180].index,inplace=True)

# Import descriptor file
path = os.getcwd()
dirname = os.path.dirname(path)
data_path = dirname+"/LogP/data/descriptors.csv"
des_df = pd.read_csv(data_path,index_col=0)

# Remove non_retained molecules
#des_df  = des_df.drop(des_df.index[[index]])
#lc_smiles_df = lc_smiles_df.drop(lc_smiles_df[[index]])
# combine two dataframe
data_comb = pd.concat([lc_smiles_df,des_df],axis=1)

In [52]:
data_comb

,SMILES,LogP,Exp_RT,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CCSC(=N)N,0.67,93.5,6.635046,0.206019,6.635046,0.206019,0.378489,104.178,96.114,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,COC(=O)N1CCN(C(=O)Cc2ccc(Cl)c(Cl)c2)[C@H](CN2C...,2.64,687.8,12.990628,-0.335181,12.990628,0.035905,0.760602,414.333,389.133,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Oc1cccc2c(O)nccc12,1.73,583.6,9.356759,-0.052407,9.356759,0.052407,0.617197,161.160,154.104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NCCc1ccc(S(=O)(=O)F)cc1,1.05,653.1,12.386283,-4.565776,12.386283,0.311786,0.740582,203.238,193.158,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CNC(=O)c1cc2c(cn1)[nH]c1ccccc12,1.33,705.6,11.531255,-0.169277,11.531255,0.169277,0.665746,225.251,214.163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,OCCn1cc(-c2ccc3c(c2)CC/C3=N\O)c(-c2ccncc2)n1,2.05,603.4,9.236427,0.039104,9.236427,0.039104,0.567510,334.379,316.235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2067,CC(C)OC(=O)N1CCC(Oc2ncnc3c2cnn3-c2ccc(S(C)(=O)...,1.59,1061.4,14.683496,-3.539697,14.683496,0.054651,0.550748,477.518,453.326,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,CCC(=O)O[C@]1(C(=O)CO)CC[C@H]2[C@@H]3CCC4=CC(=...,3.73,1059.3,12.960935,-1.190444,12.960935,0.073898,0.724147,402.531,368.259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,O=C(c1cc2ccccc2[nH]1)N1CCCC1,1.70,794.3,12.111258,0.133056,12.111258,0.133056,0.777073,214.268,200.156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
# # drop the smile column which can not be used for training
# data_comb = data_comb.drop(['SMILES'],axis=1)

# # Filling the nan with mean values in des_with_lc
# for col in data_comb:
#         data_comb[col].fillna(data_comb[col].mean(),inplace=True)
# # Remove features with low Variance(threshold <= 0.05)

# data_var = data_comb.var()
# del_feat = list(data_var[data_var <= 0.05].index)
# data_comb.drop(columns=del_feat, inplace=True)

# # Remove features with correlation(threshold >0.95)
# corr_matrix = data_comb.corr().abs()
# mask = np.triu(np.ones_like(corr_matrix,dtype=bool))
# tri_df = corr_matrix.mask(mask)
# to_drop =  [ c for c in tri_df.columns if any(tri_df[c] > 0.95)]
# data = data_comb.drop(to_drop,axis=1)

In [54]:
# parameters
n_estimators_list = [10, 50, 100, 200, 300, 400,500]
max_depth_list = range(3, 12)
min_samples_leaf_list = [1, 3, 5, 10, 20, 50]
max_features_list = ['sqrt', 'log2', 0.7, 0.8, 0.9]
min_samples_split_list = [2, 5, 10]

In [55]:
# The dataset with lc informations
#load the model with lc
path = os.getcwd()
dirname = os.path.dirname(path)

# load the model
file_path = dirname+"/logP/hyperparameters/rf_lc.pkl"
best_parameters_with_lc = pickle.load(open(file_path,'rb'))

In [56]:
#Single random run with RT
data_set_1 = des_df
data_set_2 = lc_df
des_with_lc = pd.concat([data_set_1,data_set_2],axis=1)
des_with_lc_feat_corr = des_with_lc.columns[des_with_lc.corrwith(des_with_lc['LogP']) >=0.90][:-1]
des_with_lc = des_with_lc.drop(columns=des_with_lc_feat_corr)

# Rescale the feature
def scale(col):
    return (col - np.min(col))/(np.max(col)-np.min(col))

# Filling the nan with mean values in des_with_lc
for col in des_with_lc:
    des_with_lc[col].fillna(des_with_lc[col].mean(),inplace=True)

# Remove columns with zero vlues
des_with_lc = des_with_lc.loc[:,(des_with_lc**2).sum() != 0]
data = des_with_lc.drop(['LogP'],axis=1)

# Remove features with low Variance(threshold<=0.05)
data_var = data.var()
del_feat = list(data_var[data_var <= 0.05].index)
data.drop(columns=del_feat, inplace=True)

# Remove features with correlation(threshold > 0.95)
corr_matrix = data.corr().abs()
mask = np.triu(np.ones_like(corr_matrix,dtype=bool))
tri_df = corr_matrix.mask(mask)
to_drop =  [ c for c in tri_df.columns if any(tri_df[c] > 0.95)]
data = data.drop(to_drop,axis=1)

# Scale the features
cols = list(data)
data[cols] = data[cols].apply(scale,axis=0)
data = pd.concat([data,des_with_lc['LogP']],axis=1)

# Single random run

# data set preparation
train , test = train_test_split(data,train_size = 0.5,shuffle=True)
#validate , test = train_test_split(rest, train_size = 0.5, shuffle=True)

# training set
data_tra_x = train.drop(['LogP'],axis=1)
data_tra_y = train['LogP']

# # validation set
# data_val_x = validate.drop(['LogP'],axis=1)
# data_val_y = validate['LogP']

# test set
data_tes_x = test.drop(['LogP'],axis=1)
data_tes_y = test['LogP']

best_model = RandomForestRegressor(n_estimators=n_estimators_list[best_parameters_with_lc['n_estimators']],
                                max_depth=max_depth_list[best_parameters_with_lc['max_depth']],
                                min_samples_leaf=min_samples_leaf_list[best_parameters_with_lc['min_samples_leaf']],
                                min_samples_split = min_samples_split_list[best_parameters_with_lc['min_samples_split']],
                                max_features=max_features_list[best_parameters_with_lc['max_features']],
                                min_impurity_decrease=best_parameters_with_lc['min_impurity_decrease'],
                                n_jobs=-1, random_state=1, verbose=0)

best_model.fit(data_tra_x,data_tra_y)

# training error
tra_pred = best_model.predict(data_tra_x)
tra_results = ['tra',mean_squared_error(data_tra_y,tra_pred),
            mean_squared_error(data_tra_y,tra_pred,squared=False),
            mean_absolute_error(data_tra_y,tra_pred),
            r2_score(data_tra_y,tra_pred)]

# # validation error
# val_pred = best_model.predict(data_val_x)
# val_results = ['val',mean_squared_error(data_val_y,val_pred),
#             mean_squared_error(data_val_y,val_pred,squared=False),
#             mean_absolute_error(data_val_y,val_pred),
#             r2_score(data_val_y,val_pred)]
# testing error
tes_pred = best_model.predict(data_tes_x)
tes_results = ['tes',mean_squared_error(data_tes_y,tes_pred),
            mean_squared_error(data_tes_y,tes_pred,squared=False),
            mean_absolute_error(data_tes_y,tes_pred),
            r2_score(data_tes_y,tes_pred)]


all_set = tra_results,tes_results


all_set_df = pd.DataFrame(all_set,columns=['set','mse','rmse','mae','r2'])
print(all_set_df)
#all_set_df.to_excel(dirname+"/logP/results/mlp_single_rt.xlsx")
# print('\nSingle random run with LC information is done.')

# run_best_model(des_df,lc_df)

   set       mse      rmse       mae        r2
0  tra  0.214021  0.462624  0.333134  0.925705
1  tes  0.626450  0.791486  0.597022  0.777013


In [57]:
# Predictions
predictions = pd.DataFrame(tes_pred,columns=['Predicted LogP'])

data_test = test
data_test['index'] = data_test.index

data_comb_smiles = data_comb
data_comb_smiles['index'] = data_comb_smiles.index

result = pd.merge(data_test['index'],data_comb_smiles)

test_df = result[['SMILES','LogP']]

In [58]:
test_df

,SMILES,LogP
0,CCCc1c(OCCCOc2ccc(OCC(=O)O)cc2)ccc(C(C)=O)c1O,4.15
1,O=S(=O)(c1cccc2c(I)cccc12)N1CCCNCC1,2.47
2,CNC(=O)c1c(-c2ccc(F)cc2)oc2cc(N(CCO)S(C)(=O)=O...,1.60
3,O=[N+]([O-])c1ccc(-c2cn3ccccc3n2)cc1,2.73
4,CCCCCCN1CCN(c2cccc(Cl)c2)CC1,4.75
...,...,...
1031,COc1ccc(CCNC(=O)c2ccco2)cc1OC,1.81
1032,COc1ccc2[nH]cc(C3=CCNCC3)c2c1,1.91
1033,O=C(N1CCOCC1)N1CCN(S(=O)(=O)c2cccc([N+](=O)[O-...,-0.06
1034,CN(c1ccccc1)S(=O)(=O)c1ccc(C(=O)Nc2nccs2)cc1,3.12


In [59]:
# create dataframe with predicted LogP and Experimenatal logP
predictions = pd.DataFrame(tes_pred,columns=['Predicted LogP'])

rf_df = pd.concat([test_df,predictions],axis=1)

rf_df.rename(columns={'LogP':'Experimental LogP'},inplace=True)

rf_df['Predicted LogP'] = rf_df['Predicted LogP'].round(decimals=2)


In [60]:
rf_df

,SMILES,Experimental LogP,Predicted LogP
0,CCCc1c(OCCCOc2ccc(OCC(=O)O)cc2)ccc(C(C)=O)c1O,4.15,3.52
1,O=S(=O)(c1cccc2c(I)cccc12)N1CCCNCC1,2.47,3.17
2,CNC(=O)c1c(-c2ccc(F)cc2)oc2cc(N(CCO)S(C)(=O)=O...,1.60,2.64
3,O=[N+]([O-])c1ccc(-c2cn3ccccc3n2)cc1,2.73,2.02
4,CCCCCCN1CCN(c2cccc(Cl)c2)CC1,4.75,4.17
...,...,...,...
1031,COc1ccc(CCNC(=O)c2ccco2)cc1OC,1.81,1.75
1032,COc1ccc2[nH]cc(C3=CCNCC3)c2c1,1.91,2.31
1033,O=C(N1CCOCC1)N1CCN(S(=O)(=O)c2cccc([N+](=O)[O-...,-0.06,1.06
1034,CN(c1ccccc1)S(=O)(=O)c1ccc(C(=O)Nc2nccs2)cc1,3.12,3.24


In [61]:
# Find deviation
rf_df['Deviation'] = (rf_df['Experimental LogP']- rf_df['Predicted LogP'])

In [62]:
# round off decimal value
rf_df['Deviation'] = rf_df['Deviation'].round(decimals=2)

In [63]:
rf_df

,SMILES,Experimental LogP,Predicted LogP,Deviation
0,CCCc1c(OCCCOc2ccc(OCC(=O)O)cc2)ccc(C(C)=O)c1O,4.15,3.52,0.63
1,O=S(=O)(c1cccc2c(I)cccc12)N1CCCNCC1,2.47,3.17,-0.70
2,CNC(=O)c1c(-c2ccc(F)cc2)oc2cc(N(CCO)S(C)(=O)=O...,1.60,2.64,-1.04
3,O=[N+]([O-])c1ccc(-c2cn3ccccc3n2)cc1,2.73,2.02,0.71
4,CCCCCCN1CCN(c2cccc(Cl)c2)CC1,4.75,4.17,0.58
...,...,...,...,...
1031,COc1ccc(CCNC(=O)c2ccco2)cc1OC,1.81,1.75,0.06
1032,COc1ccc2[nH]cc(C3=CCNCC3)c2c1,1.91,2.31,-0.40
1033,O=C(N1CCOCC1)N1CCN(S(=O)(=O)c2cccc([N+](=O)[O-...,-0.06,1.06,-1.12
1034,CN(c1ccccc1)S(=O)(=O)c1ccc(C(=O)Nc2nccs2)cc1,3.12,3.24,-0.12


In [64]:
# dataframe to csv
rf_df.to_csv('rf_exp_pre.csv')

In [65]:
# The three most deviation values
rf_df.iloc[list(rf_df['Deviation'].nlargest(3).index)]

,SMILES,Experimental LogP,Predicted LogP,Deviation
855,CC(C)c1cc(C(F)(F)F)nc(NNc2nc(C(C)C)cc(C(F)(F)F...,6.55,3.52,3.03
625,Cc1nc(N2CCN(c3ccccn3)CC2)c2c(C)c(C(=O)Nc3ccc(F...,5.84,2.90,2.94
902,COc1cc(-c2ccc(Nc3ccccc3C(=O)O)c(OC)c2)ccc1Nc1c...,8.10,5.19,2.91


In [67]:
rf_df[rf_df['Deviation']==3.03]['SMILES'].item()

'CC(C)c1cc(C(F)(F)F)nc(NNc2nc(C(C)C)cc(C(F)(F)F)n2)n1'

In [68]:
import pubchempy

smiles ='CC(C)c1cc(C(F)(F)F)nc(NNc2nc(C(C)C)cc(C(F)(F)F)n2)n1'
compounds = pubchempy.get_compounds(smiles, namespace='smiles')
match = compounds[0]
print(match.iupac_name)

1,2-bis[4-propan-2-yl-6-(trifluoromethyl)pyrimidin-2-yl]hydrazine


In [69]:
rf_df[rf_df['Deviation']==2.94]['SMILES'].item()

'Cc1nc(N2CCN(c3ccccn3)CC2)c2c(C)c(C(=O)Nc3ccc(F)cc3F)sc2n1'

In [70]:
smiles ='Cc1nc(N2CCN(c3ccccn3)CC2)c2c(C)c(C(=O)Nc3ccc(F)cc3F)sc2n1'
compounds = pubchempy.get_compounds(smiles, namespace='smiles')
match = compounds[0]
print(match.iupac_name)

N-(2,4-difluorophenyl)-2,5-dimethyl-4-(4-pyridin-2-ylpiperazin-1-yl)thieno[2,3-d]pyrimidine-6-carboxamide


In [71]:
rf_df[rf_df['Deviation']==2.91]['SMILES'].item()

'COc1cc(-c2ccc(Nc3ccccc3C(=O)O)c(OC)c2)ccc1Nc1ccccc1C(=O)O'

In [72]:
smiles ='CCOc1cc(-c2ccc(Nc3ccccc3C(=O)O)c(OC)c2)ccc1Nc1ccccc1C(=O)O'
compounds = pubchempy.get_compounds(smiles, namespace='smiles')
match = compounds[0]
print(match.iupac_name)

None
